In [2]:
# Pour lire les données
from bs4 import BeautifulSoup
import pandas as pd

# Pour le prétraitement des données
# Token et filtre et lemma
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Pour analyse de sentiments
from textblob import TextBlob

### Lecture des données

In [3]:
with open('../data/brexit.txt', 'r', encoding='utf-8') as file:
        brexit = file.read()

In [4]:
soup = BeautifulSoup(brexit, 'html.parser') 
texts = soup.find_all('text')

brexit_data = pd.DataFrame(columns=['id', 'title', 'author', 'year', 'date', 'section', 'sourcename', 'text'])

for text in texts:
    text_data = {
        'id': text.get('id'),
        'title': text.get('title'),
        'author': text.get('author'),
        'year': text.get('year'),
        'date': text.get('date'),
        'section': text.get('section'),
        'sourcename': text.get('sourcename'),
        'text': text.get_text(separator=" ", strip=True),
        'polarity': text.get('polarity')
    }
    brexit_data = pd.concat([brexit_data, pd.DataFrame([text_data])], ignore_index=True)



In [5]:
brexit_data

,id,title,author,year,date,section,sourcename,text
0,1,Economie & EntrepriseLe risque grec fait de no...,NULL,2015,07/01/2015,NULL,Le Monde,Economie & EntrepriseLe risque grec fait de no...
1,2,David Cameron donne des gages aux eurosceptiqu...,Philippe Bernard,2015,07/01/2015,NULL,Le Monde,David Cameron donne des gages aux eurosceptiqu...
2,3,Merkel calme le jeu avec Athènes et Londres,"Philippe Bernard, Frédéric Lemaître et Jean-Pi...",2015,09/01/2015,NULL,Le Monde,Merkel calme le jeu avec Athènes et Londres Lo...
3,4,"2015, une année électorale qui va stresser les...",NULL,2015,19/01/2015,NULL,Les Echos,"2015, une année électorale qui va stresser les..."
4,5,La croissance britannique face au doute électoral,"Collomp, Florentin; florentin collomp @fcollom...",2015,28/01/2015,NULL,Le Figaro,La croissance britannique face au doute électo...
...,...,...,...,...,...,...,...,...
8669,8670,Les marchés plutôt optimistes pour 2017,Emmanuel Schafroth,2016,31/12/2016,NULL,Le Monde,Les marchés plutôt optimistes pour 2017 Le pay...
8670,8671,"Depuis quinze ans, l'euro dans les porte-monna...",NULL,2016,31/12/2016,NULL,Midi Libre - CARCA_ML,"Depuis quinze ans, l'euro dans les porte-monna..."
8671,8672,Éditorial par François Régis Hutin,NULL,2016,31/12/2016,NULL,Ouest-France,Éditorial par François Régis Hutin Tout ou pre...
8672,8673,[C'est sous la présidence maltaise de l'Union...],NULL,2016,31/12/2016,NULL,Ouest-France,[C'est sous la présidence maltaise de l'Union....


### Tokenisation

#### Ressource pour NLTK :

In [6]:
nltk.download('punkt')   # Pour la tokenisation

nltk.download('stopwords')  # Pour els mots d'arrêts

nltk.download('wordnet')  # Pour Lemmatisaion


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mbrei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mbrei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mbrei\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
brexit_token = [word_tokenize(texte) for texte in brexit_data['text']]

In [8]:
len(brexit_token[0])

168

### Supression mot d'arrêt

In [9]:
mots_arret = set(stopwords.words('french'))

brexit_filtre = []
for article in brexit_token:
    article_filtre = [mot for mot in article if mot.lower() not in mots_arret]
    brexit_filtre.append(article_filtre)

In [10]:
brexit_filtre[1][10]

"L'immigration"

### Lemmatisation

In [11]:
lemma = WordNetLemmatizer()

brexit_lemma = []
for article in brexit_filtre:
    article_lemma = [lemma.lemmatize(mot) for mot in article]
    brexit_lemma.append(article_lemma)

In [12]:
len(brexit_lemma[0])

115

### JOIN

In [13]:
brexit_full=[]
for article in brexit_lemma:
    brexit_full.append(' '.join(article))

In [14]:
brexit_full[0]

"Economie & EntrepriseLe risque grec fait nouveau trembler zone euro trois . Après turbulence mi-octobre 2014 , puis décembre , Bourses mondiales connu nouvel accès faiblesse , lundi 5 janvier . principaux index européens asiatiques perdu entre 3 % 5 % , Wall Street a clôturé rouge , sillage Bourse d'Athènes ( - 5,63 % ) . confirmation craintes observateurs 2015 : l'année placée sou signe nervosité Bourse . raisons faire peur devraient manquer : perspective élections législatives grecques , 25 janvier , puis scrutin britannique mai , prélude référendum éventuel « Brexit » ( sortie Royaume-Uni l'UE ) . Sans oublier , fin d'année , élections Espagne , autre maillon faible zone euro ..."

# Analyse de sentiments

### Polarité, caractère positif ou négatif

#### Avec texte de base

In [15]:
brexit_polarite = []
for article in brexit_data["text"]:
    article_pola = TextBlob(article).sentiment.polarity
    brexit_polarite.append(article_pola)

In [1]:
brexit_polarite

NameError: name 'brexit_polarite' is not defined

In [17]:
import statistics
statistics.mean(brexit_polarite)

0.05005946435161881

#### Avec article transformé

In [18]:
brexit_full_polarite = []
for article in brexit_full:
    article_pola = TextBlob(article).sentiment.polarity
    brexit_full_polarite.append(article_pola)

In [19]:
statistics.mean(brexit_full_polarite)

0.06117555505216972

### Subjectivité

In [20]:
brexit_subj = []
for article in brexit_data["text"]:
    article_pola = TextBlob(article).sentiment.subjectivity
    brexit_subj.append(article_pola)

In [21]:
brexit_full_subj = []
for article in brexit_full:
    article_pola = TextBlob(article).sentiment.subjectivity
    brexit_full_subj.append(article_pola)

In [22]:
#Nombre d'article dont la subjectivité est inférieure à 0.2
len([x for x in brexit_full_subj if x <0.2])

2653

In [23]:
print(statistics.mean(brexit_subj), statistics.mean(brexit_full_subj))

0.3930188767622335 0.4047668149168394


# Analyse de sentiments par Journal

In [24]:
brexit_data.groupby('sourcename')
brexit_data['sourcename'].value_counts()

sourcename
Les Echos                           1781
Le Figaro                           1475
Le Monde                            1298
La Croix                             458
Libération                           427
                                    ... 
Sud Ouest - Landes                     1
Le Progrès - Lyon - 42G                1
Ouest-France - Vendee Littoral         1
Le Progrès - Lyon - 42F                1
Le Progrès - Lyon - Est Lyonnais       1
Name: count, Length: 195, dtype: int64

### Subjectivité 

In [27]:
journal = ['Les Echos', 'Le Monde', 'Le Figaro', 'Libération','La Croix']
subjectivite = []

for i in range(len(journal)):
    data = brexit_data.groupby('sourcename').get_group(journal[i])
    brexit_subj = []
    for article in data["text"]:
        article_subj = TextBlob(article).sentiment.subjectivity
        brexit_subj.append(article_subj)
    subj_mean = statistics.mean(brexit_subj)
    subjectivite.append(subj_mean)
    print(i)

0
1
2
3
4


In [28]:
pd.DataFrame({'Journal':journal, 'Subjectivité moyenne':subjectivite})

,Journal,Subjectivité moyenne
0,Les Echos,0.419597
1,Le Monde,0.437484
2,Le Figaro,0.397718
3,Libération,0.456161
4,La Croix,0.351072


# Classification